In [1]:
import cv2 as cv
import imghdr
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential

2023-05-17 22:51:17.742407: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = r"/home/ec2-user/dataset/train"
img_exts = ["jpeg", "jpg", "png"]

In [3]:
X = []
y = []

for img_class in os.listdir(data_dir):
    for img in os.listdir(os.path.join(data_dir, img_class)):
        img_path = os.path.join(data_dir, img_class, img)
        try:
            img_read = cv.imread(img_path)
            _type = imghdr.what(img_path)
            if _type not in img_exts:
                print("img extension not in img_exts {}".format(img_path))
                os.remove(img_path)
            X.append(cv.resize(img_read, (180, 180)).flatten())
            y.append(img_class)
        except Exception as e:
            print("issue with image {}".format(img_path))

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer

le = LabelEncoder()
y = le.fit_transform(y)
list(set(y))

[0, 1, 2]

In [5]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingClassifier


bag_tree = BaggingClassifier(DecisionTreeClassifier(max_features='sqrt'), n_estimators=2, random_state=42)
bag_knn = BaggingClassifier(KNeighborsClassifier(n_neighbors=3), n_estimators=2, random_state=42)

voting_clf = VotingClassifier(
 estimators=[('lr', bag_tree), ('rf', bag_knn)],
 voting='hard')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              BaggingClassifier(estimator=DecisionTreeClassifier(max_features='sqrt'),
                                                n_estimators=2,
                                                random_state=42)),
                             ('rf',
                              BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=3),
                                                n_estimators=2,
                                                random_state=42))])

In [ ]:
y_pred = voting_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

In [ ]:
from keras.preprocessing import image
 
img_test = cv.imread(r"C:\Users\victor.v.casimiro\Documents\faculdade\TCC\dataset\test\Banana_Good\IMG_8560.jpg")
img_test = cv.resize(img_test, (180, 180)).flatten()
voting_clf.predict([img_test])

In [ ]:
img_test = cv.imread(r"C:\Users\victor.v.casimiro\Documents\faculdade\TCC\dataset\test\Orange_Good\IMG_20190924_150951.jpg")
img_test = cv.resize(img_test, (180, 180)).flatten()
voting_clf.predict([img_test])

In [ ]:
img_test = cv.imread(r"C:\Users\victor.v.casimiro\Documents\faculdade\TCC\dataset\test\Apple_Good")
img_test = cv.resize(img_test, (180, 180)).flatten()
voting_clf.predict(img_test)